In [ ]:
!pip install ydata-profiling

In [ ]:
# Importamos las librerias
import pandas as pd
from ydata_profiling import ProfileReport

In [ ]:
# cargamos el dataset
try:
  df = pd.read_csv('/content/Sample Sales Data.csv', encoding= "latin-1")
except FileNotFoundError:
  print("Error: Asegúrate de que 'Sample Sales Data.csv' esté en el directorio correcto")
  exit()

df.shape

In [ ]:
# Generamos el reporte de Pandas Profiling
print("Generando el reporte de Pandas Profiling...")
profile = ProfileReport(
    df,
    title="Reporte de Exploración de Datos de Ventas",
    html={'style': {'full_width': True}},
    infer_dtypes=False,
    minimal=True
)

# Guardamos el reporte en formato HTML
profile.to_file("reporte_ventas.html")

print("Reporte de Pandas Profiling generado exitosamente.")

# Mostramos la información inicial de DataFrame
print("Información inicial del DataFrame:")
df.info

# Mostramos las primeras 5 filas para verificación
print("\nPrimeras 5 filas del dataset:")
print(df.head())

# I. Descripción de Necesidades de Limpieza y Estandarización

Tras realizar la exploración inicial de datos (EDA) utilizando *Pandas Profiling* e inspección manual, se ha establecido el siguiente diagnóstico de calidad de datos. A continuación, se detallan las intervenciones necesarias para transformar el dataset crudo en un activo analítico confiable.

## 1. Diagnóstico de Calidad de Datos

###  Duplicados
* **Observación:** Se realizó una validación de duplicados exactos en la totalidad de los registros.
* **Hallazgo:** El dataset presenta un **0% de filas duplicadas**.
* **Acción:** Aunque no se requiere eliminación masiva, se ejecutará el comando `drop_duplicates()` como medida preventiva estándar para asegurar la integridad en futuras ejecuciones.

###  Valores Nulos (Missing Values)
Se identificaron valores nulos con implicaciones críticas:

1.  **Nulos en Variables**
    * **`ADDRESSLINE2`:** Presenta un **95.1% de valores nulos**. Al no aportar valor analítico y tener una tasa de ausencia tan alta, se procederá a su **eliminación**.
    * **`STATE`, `POSTALCODE`, `TERRITORY`:** Presentan nulos en registros válidos (especialmente en ventas internacionales fuera de EE. UU.).
    * *Acción:* **Imputación categórica**. Se rellenarán con el valor estandarizado `'Unknown'` o `'NA'` para preservar la transacción sin perder la información geográfica disponible (País).

###  Inconsistencias en Valores
* **Variable `STATUS`:** La columna incluye estados operativos como *'Cancelled'*, *'On Hold'* o *'Disputed'*.
* **Impacto:** Incluir estas órdenes inflaría artificialmente las métricas de ingresos, ya que no representan ventas concretadas.
* **Acción:** Se aplicará un **filtro estricto** para conservar únicamente las transacciones con estado **'Shipped'** o **'Resolved'**.

###  Tipos de Datos (Casting)
Se detectaron discrepancias entre el tipo de dato inferido y la naturaleza semántica de la variable:
* **`ORDERDATE`:** Actualmente clasificada como *Object* (String). Es imperativo convertirla a **`datetime`** para habilitar el análisis de series temporales (Año, Trimestre, Mes).
* **Variables Categóricas:** Columnas como `PRODUCTLINE`, `COUNTRY` y `DEALSIZE` se convertirán al tipo **`category`** para optimizar el uso de memoria y facilitar el análisis estadístico.

###  Valores Atípicos (Outliers)
* **Observación:** La variable **`SALES`** presenta una distribución con sesgo positivo (*right-skewed*).
* **Análisis:** Los valores extremos superiores corresponden a transacciones con `DEALSIZE` = *'Large'*.
* **Decisión:** Estos valores **no se eliminarán**, ya que representan ventas legítimas de alto valor ("High-Ticket Sales") que son críticas para el análisis de ingresos. Se documentará su existencia.

###  Nivel de Granularidad y Agregación
* **Estado Actual:** Granularidad a nivel de **Línea de Pedido** (cada fila es un producto dentro de una orden).
* **Requerimiento:** El objetivo del negocio es analizar tendencias macroeconómicas y desempeño por segmentos.
* **Acción:** Se generará un nuevo dataset agregado agrupando por:
    * `YEAR_ID`, `QTR_ID` (Temporalidad)
    * `COUNTRY` (Geografía)
    * `PRODUCTLINE` (Producto)
* **Métricas a Calcular:** Suma total de `SALES` y conteo de `ORDERNUMBER`.

---
*Este plan de limpieza garantiza que el análisis posterior se base exclusivamente en datos íntegros, transacciones exitosas y formatos temporales correctos.*

In [ ]:
# Pandas Profiling nos arrojo que no hay duplicados. limpinamos Duplicados, por buenas practicas

# Creamos una copia del DataFrame
df_cleaned = df.copy()

# revisamos si tenemos valores duplicados
duplicate_rows = df_cleaned.duplicated().sum()

print(f"Se encontraron {duplicate_rows} duplicados en el dataset.")

In [ ]:
# ajustar el tipo de dato

# Convertimos ORDERDATE en un datetime
df_cleaned['ORDERDATE'] = pd.to_datetime(df_cleaned['ORDERDATE'])

# Convertir columnas de ID a números enteros, utilizando el tipo de dato Int64 de pandas que permite valores nulos
for col in ['QTR_ID', 'MONTH_ID', 'YEAR_ID']:
    df_cleaned[col] = df_cleaned[col].astype('Int64')

# Conviertimos las columnas categóricas claves al tipo 'category' para ahorrar memoria
for col in ['STATUS', 'PRODUCTLINE', 'DEALSIZE', 'COUNTRY']:
    df_cleaned[col] = df_cleaned[col].astype('category')

print("✓ Data types have been successfully adjusted.")
display(df_cleaned.info())

In [ ]:
# columna eliminada 95% de datos nulos
df_cleaned = df_cleaned.drop('ADDRESSLINE2', axis=1)
print("Columna 'ADDRESSLINE2' eliminada (95% nula).")

In [ ]:
# gestionamos valores nulos
df_cleaned['STATE'].fillna('Unknown', inplace=True)
df_cleaned['POSTALCODE'].fillna('Unknown', inplace=True)
df_cleaned['TERRITORY'].fillna('Unknown', inplace=True)

In [ ]:
# Calculamos el número de valores nulos y su porcentaje para cada columna
null_counts = df_cleaned.isnull().sum()
null_percentages = (null_counts / len(df_cleaned) * 100).round(2)
null_summary = pd.DataFrame({'Null Count': null_counts, 'Null Percentage (%)': null_percentages})

# Filtrar por columnas
null_summary = null_summary[null_summary['Null Count'] > 0]

print("Columns with Null Values:")
display(null_summary)

In [ ]:
import plotly.express as px
import pandas as pd

# datos atipicos
# Crea una figura para los diagramas de caja.
fig = px.box(df_cleaned,
             y=['SALES', 'QUANTITYORDERED', 'PRICEEACH'],
             title='Boxplots for Outlier Detection',
             facet_col="variable",
             facet_col_wrap=3,
             color_discrete_sequence=['#3ea7fb'])

# Update layout for better readability
fig.update_layout(
    height=450,
    width=800,
    showlegend=False,
    yaxis_title=None
)
fig.for_each_yaxis(lambda y: y.update(showticklabels=True, title=''))
fig.update_traces(showlegend=False)

fig.show()

In [ ]:
# Aggregate data to get monthly sales
# Extract year and month from ORDERDATE
df_cleaned['ORDER_YEAR_MONTH'] = df_cleaned['ORDERDATE'].dt.to_period('M')

# Group by the new month-year column and sum the sales
monthly_sales = df_cleaned.groupby('ORDER_YEAR_MONTH')['SALES'].sum().reset_index()
monthly_sales['ORDER_YEAR_MONTH'] = monthly_sales['ORDER_YEAR_MONTH'].astype(str)

print("Aggregated Monthly Sales:")
display(monthly_sales)

In [ ]:
df_agregado = df.groupby(['YEAR_ID', 'QTR_ID', 'COUNTRY', 'PRODUCTLINE']).agg(
    Total_Sales=('SALES', 'sum'),
    Conteo_Ordenes=('ORDERNUMBER', 'count'),
    Venta_Promedio=('SALES', 'mean')
).reset_index()

print("----------------------------------------")
print("PROCESO DE LIMPIEZA COMPLETADO.")
print(f"Dataset de línea de pedido reducido a {len(df)} filas.")
print(f"Dataset final AGREGADO a granularidad de análisis: {len(df_agregado)} filas.")
print("Columnas del dataset agregado:", df_agregado.columns.tolist())

In [ ]:
df_agregado.head()

In [ ]:
# descargo el DataFrame agregado con los datos limpios
# Definir el nombre del archivo de salida
nombre_archivo = 'Ventas_Globales_Limpias_Agregadas.csv'

# Exportar a CSV
# index=False es IMPORTANTE: evita que se guarde una columna extra con los números de fila (0, 1, 2...)
df_agregado.to_csv(nombre_archivo, index=False)

print(f" Archivo '{nombre_archivo}' generado exitosamente.")
print("Busca este archivo en la misma carpeta donde está guardado tu Notebook.")

In [ ]:
print("Null values have been handled. Remaining nulls:", df_agregado.isnull().sum().sum())

## III. Validación del Dataset Limpio

La etapa de validación confirma que el conjunto de datos resultante, **df_agregado**, es apto y suficiente para responder a los objetivos de análisis planteados en la Evidencia de Aprendizaje 1, a pesar de las transformaciones y la eliminación de filas nulas.

| Aspecto de Validación | Pregunta Clave | Estado Post-Limpieza | Confirmación |
| :--- | :--- | :--- | :--- |
| **Pregunta de Negocio** | ¿El dataset contiene las variables `YEAR_ID`, `COUNTRY` y `PRODUCTLINE` con las ventas agregadas? | **VALIDADO** | La agregación creó la columna `Total_Sales` a la granularidad requerida, lista para el análisis de tendencias. |
| **Completitud de Datos** | ¿Se han eliminado todos los valores nulos en las columnas críticas? | **VALIDADO** | Se eliminaron los 1,315 registros vacíos, y los nulos restantes en `STATE`/`TERRITORY` fueron imputados categóricamente como 'Unknown', resultando en **cero nulos** en el dataset de análisis. |
| **Relevancia de Variables** | ¿Se mantienen las variables esenciales para el objetivo? | **VALIDADO** | Se mantuvieron todas las variables necesarias y se eliminaron solo variables auxiliares (`ADDRESSLINE2`), manteniendo el foco en el desempeño comercial. |
| **Granularidad Adecuada** | ¿El nivel de detalle es apropiado para el análisis de tendencias? | **VALIDADO** | La granularidad se ajustó del nivel de **Línea de Pedido** al nivel **Segmentado (Año, Trimestre, País, Producto)**, perfecto para el análisis descriptivo estratégico. |

**Conclusión:** El dataset se encuentra en un estado **Óptimo** para proceder con la fase de Análisis Descriptivo y Visualización.